# Analyzing Olympics Data

In [ ]:
# Import libraries
import pandas as pd
import plotly.express as px

# Import the data
olympics = pd.read_csv("athlete_events.csv")

# Preview the DataFrame
olympics

In [2]:
# Inspect the DataFrame
olympics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      271116 non-null  int64  
 1   name    271116 non-null  object 
 2   sex     271116 non-null  object 
 3   age     261642 non-null  float64
 4   height  210945 non-null  float64
 5   weight  208241 non-null  float64
 6   team    271116 non-null  object 
 7   noc     271116 non-null  object 
 8   games   271116 non-null  object 
 9   year    271116 non-null  int64  
 10  season  271116 non-null  object 
 11  city    271116 non-null  object 
 12  sport   271116 non-null  object 
 13  event   271116 non-null  object 
 14  medal   39783 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB


In [3]:
# Check missing values
olympics.isna().sum()

id             0
name           0
sex            0
age         9474
height     60171
weight     62875
team           0
noc            0
games          0
year           0
season         0
city           0
sport          0
event          0
medal     231333
dtype: int64

There are many missing values in the `medal` column because the dataset contains all competitors including those who did not win a medal.

The 'team' column contains hyphens and backslashes because some players have multiple country affiliations.

In [ ]:
# Inspect the team column
olympics["team"].value_counts().to_frame()

In [5]:
# Split the team column on forward slashes and hyphens
olympics["team_clean"] = olympics["team"].str.split("[/-]").str[0]

# Preview the new column
olympics["team_clean"].unique()

array(['China', 'Denmark', 'Netherlands', 'United States', 'Finland',
       'Norway', 'Romania', 'Estonia', 'France', 'Taifun', 'Morocco',
       'Spain', 'Egypt', 'Iran', 'Bulgaria', 'Italy', 'Chad',
       'Azerbaijan', 'Sudan', 'Russia', 'Argentina', 'Cuba', 'Belarus',
       'Greece', 'Cameroon', 'Turkey', 'Chile', 'Mexico', 'Soviet Union',
       'Nicaragua', 'Hungary', 'Nigeria', 'Algeria', 'Kuwait', 'Bahrain',
       'Pakistan', 'Iraq', 'United Arab Republic', 'Lebanon', 'Qatar',
       'Malaysia', 'Germany', 'Thessalonki', 'Canada', 'Ireland',
       'Australia', 'South Africa', 'Eritrea', 'Tanzania', 'Jordan',
       'Tunisia', 'Libya', 'Belgium', 'Djibouti', 'Palestine', 'Comoros',
       'Kazakhstan', 'Brunei', 'India', 'Saudi Arabia', 'Syria',
       'Maldives', 'Ethiopia', 'United Arab Emirates', 'North Yemen',
       'Indonesia', 'Philippines', 'Singapore', 'Uzbekistan',
       'Kyrgyzstan', 'Tajikistan', 'Unified Team', 'Japan',
       'Congo (Brazzaville)', 'Switzerlan

## Combine data from other sources to include population data

In [6]:
SELECT 
      name AS country,
	  year,
	  population
FROM nation.countries
INNER JOIN nation.country_stats USING(country_id)

country  year  population
0        Aruba  1986       62644
1        Aruba  1987       61833
2        Aruba  1988       61079
3        Aruba  1989       61032
4        Aruba  1990       62149
...        ...   ...         ...
9509  Zimbabwe  2014    13586681
9510  Zimbabwe  2015    13814629
9511  Zimbabwe  2016    14030390
9512  Zimbabwe  2017    14236745
9513  Zimbabwe  2018    14439018

[9514 rows x 3 columns]

In [7]:
# Perform a left join between the two DataFrames
olympics_new = olympics.merge(nations_data,
                             left_on=["team_clean", "year"],
                             right_on=["country", "year"],
                             how = "left")

# Preview our data
olympics_new

id                      name  ...      country    population
0            1                 A Dijiang  ...        China  1.164970e+09
1            2                  A Lamusi  ...        China  1.350695e+09
2            3       Gunnar Nielsen Aaby  ...          NaN           NaN
3            4      Edgar Lindenau Aabye  ...          NaN           NaN
4            5  Christine Jacoba Aaftink  ...  Netherlands  1.476009e+07
...        ...                       ...  ...          ...           ...
271111  135569                Andrzej ya  ...          NaN           NaN
271112  135570                  Piotr ya  ...       Poland  3.801174e+07
271113  135570                  Piotr ya  ...       Poland  3.801174e+07
271114  135571        Tomasz Ireneusz ya  ...       Poland  3.866348e+07
271115  135571        Tomasz Ireneusz ya  ...       Poland  3.823036e+07

[271116 rows x 18 columns]

## 1. Which countries have the most gold medals?

In [ ]:
# Count the number of gold medals earned by a country
gold_total = olympics_new.query("medal == 'Gold'").groupby("team_clean", as_index = False)["medal"].count()

# Sort the values
gold_total.sort_values(by="medal", ascending = False, inplace=True)

# Preview our count
gold_total

In [ ]:
# Create choropleth map of gold medal counts
fig = px.choropleth(
  gold_total,
    locations="team_clean",
    locationmode="country names",
    color="medal",
    labels={"team_clean": "Country", "medal": "Medal Count"},
    title="Number of Gold Medals by Country"
)

fig.show()

## 2. Did the number of sports grow over time?

In [ ]:
# Group by year and season and count the number of unique values
sports_category = olympics_new.groupby(["year","season"], as_index=False)["sport"].nunique()

# Preview the DataFrame
sports_category

In [ ]:
# Create a line plot for Summer and Winter Olympics
line_plot = px.line(
  sports_category,
    x="year",
    y="sport",
    color="season",
    labels={"year": "Year", "season": "Season", "sport": "Sport Count"},
    title="Did the Number of Sports Increase in Olympics?"
)

line_plot.show()

## 3. Which countries had the highest medal count per 10 million people in 2016?

In [ ]:
# Calculate event medals
total_medals = olympics_new.query("year == 2016")\
.groupby(["team_clean","event","medal","population"], as_index=False)["medal"].first()

# Preview the DataFrame
total_medals

In [ ]:
# Group by the team and population
medal_counts =total_medals.groupby(["team_clean", "population"], as_index=False)["medal"].count()

# Calculate the number of medals per 10000000 people
medal_counts["per_10m"] = medal_counts["medal"] / (medal_counts["population"] / 10000000)

# Sort values and take the top 20 countries
top_countries = medal_counts.sort_values(by = "per_10m", ascending = False).head(20)

# Preview the DataFrame
top_countries

In [ ]:
# Create a column chart by medal per capita
fig = px.bar(
top_countries,
    x="per_10m",
    y="team_clean",
    labels={"team_clean": "Country", "per_10m": "Number of medals per 10 million population"},
    hover_data=["population"]
)

fig.show()